In [1]:
import os
import json
from pprint import pprint

# List to hold the data from all JSON files
all_json_data = {}

# Get the current directory
current_directory = os.getcwd()

# Loop through all files in the current directory
for filename in os.listdir(current_directory):
    if filename.endswith('.json'):
        # if filename.startswith('.'):
            # continue
        file_path = os.path.join(current_directory, filename)
        
        # Open and read the JSON file
        with open(file_path, 'r') as file:
            try:
                json_data = json.load(file)

                label = filename.split('.')[0]
                if len(label) == 0:
                    continue

                all_json_data[label] = json_data
            except json.JSONDecodeError as e:
                print(f"Error reading {filename}: {e}")

# Now, all_json_data contains the contents of all JSON files
pprint(all_json_data.keys())


dict_keys(['openai_rabi_failure_cases', 'openai_gmm_failure_cases', 'openai_drag_success_cases', 'openai_gmm_success_cases', 'openai_drag_failure_cases', 'openai_rabi_success_cases', 'openai_resonator_spec_failure_cases', 'openai_resonator_spec_success_cases'])


In [2]:
all_json_data.keys()

dict_keys(['openai_rabi_failure_cases', 'openai_gmm_failure_cases', 'openai_drag_success_cases', 'openai_gmm_success_cases', 'openai_drag_failure_cases', 'openai_rabi_success_cases', 'openai_resonator_spec_failure_cases', 'openai_resonator_spec_success_cases'])

In [3]:
import pandas as pd

In [4]:
entries = []

def summarize_statistics(data):

    entries = []
    for key,val in data.items():
        data = {
            'model_type':key
        }
        data.update(val)
        entries.append(data)

    return entries

for case,data in all_json_data.items():
    model = case.split('_')[0]
    case = '_'.join(case.split('_')[1:-1])
    for _, report in data.items():
        for inspection_type, results in report.items():
            entry = {}
            entry['inspection_type']=inspection_type
            entry['case'] = case
            entry['model'] = model
            for r in summarize_statistics(results):
                r.update(entry)
                entries.append(r)

In [5]:
print(entries[0].keys())

dict_keys(['model_type', 'plot_fft', 'fitting', 'Final analysis', 'Suggested parameter updates', 'Experiment success', 'inspection_type', 'case', 'model'])


In [6]:
df = pd.DataFrame(entries)

In [7]:
df

,model_type,plot_fft,fitting,Final analysis,Suggested parameter updates,Experiment success,inspection_type,case,model,plot_hexbin,plot,plot_magnitude
0,full,The plot shows the frequency spectrum of Rabi ...,The fitting result of the Rabi oscillation sug...,The experiment successfully observed Rabi osci...,amp should be updated by 0.20677828,True,no_image_few_shot,rabi_failure,openai,NaN,NaN,NaN
1,fitting_only,NaN,The fitting result of the Rabi oscillation sug...,The experiment successfully demonstrated a cle...,amp should be updated by 0.20677828,True,no_image_few_shot,rabi_failure,openai,NaN,NaN,NaN
2,visual_only,The plot shows the frequency spectrum of Rabi ...,NaN,The experiment successfully observed Rabi osci...,None,True,no_image_few_shot,rabi_failure,openai,NaN,NaN,NaN
3,full,The plot shows the frequency spectrum of Rabi ...,The fitting result of the Rabi oscillation sug...,The experiment was successful as indicated by ...,amp should be updated by 0.15745233,True,image_few_shot,rabi_failure,openai,NaN,NaN,NaN
4,fitting_only,NaN,The fitting result of the Rabi oscillation sug...,The experiment results indicate a successful R...,amp should be updated by 0.15745233,True,image_few_shot,rabi_failure,openai,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
955,fitting_only,NaN,The fitting suggest that the resonant frequenc...,The experiment successfully detected a resonat...,None,True,no_image_few_shot,resonator_spec_success,openai,NaN,NaN,NaN
956,visual_only,NaN,NaN,The experiment successfully identified a reson...,None,True,no_image_few_shot,resonator_spec_success,openai,NaN,NaN,The resonator spectroscopy magnitude plot show...
957,full,NaN,The fitting suggest that the resonant frequenc...,The experiment successfully detected a resonat...,None,True,image_few_shot,resonator_spec_success,openai,NaN,NaN,The resonator spectroscopy magnitude plot show...
958,fitting_only,NaN,The fitting suggest that the resonant frequenc...,The experiment successfully detected a resonat...,None,True,image_few_shot,resonator_spec_success,openai,NaN,NaN,NaN


In [8]:
mask = df['case'].str.contains('success')
df['target_output'] = mask
df['ai_output'] = df['Experiment success']
df['success'] = (df['ai_output'] == df['target_output'])

In [10]:
# get the ratio of True in "full success"
# group by the value of "case"
df.groupby(['case','model_type','inspection_type'])['success'].mean()

case                    model_type    inspection_type  
drag_failure            fitting_only  image_few_shot       0.55
                                      no_image_few_shot    0.60
                        full          image_few_shot       0.55
                                      no_image_few_shot    0.60
                        visual_only   image_few_shot       0.60
                                      no_image_few_shot    0.75
drag_success            fitting_only  image_few_shot       0.95
                                      no_image_few_shot    1.00
                        full          image_few_shot       1.00
                                      no_image_few_shot    1.00
                        visual_only   image_few_shot       0.70
                                      no_image_few_shot    0.90
gmm_failure             fitting_only  image_few_shot       0.00
                                      no_image_few_shot    0.00
                        full          image_few_